In [ ]:
model_str = "GNN"  # or "GNN"

import matplotlib.pyplot as plt
from data_loading import load_dataset
import torch
from deepcardio.losses import LpLoss
from predict import ModelInference
import numpy as np
from raw_data_handling import single_case_handling

data_path = "/mnt/research/compbiolab/Ehsan/DeepCardioSim/cardiac_models/electrophysio/data_processed"
save_dir = f'./{model_str}/ckpt/'

if model_str == "GINO":
    from GINO.model import initialize_GINO_model
    model = initialize_GINO_model(n_fno_modes=16)
    
elif model_str == "GNN":
    from GNN.model import initialize_GNN_model
    model = initialize_GNN_model(size_hidden_layers=64)

else:
    raise ValueError("Only 'GINO' or 'GNN' can be passed.")

dltrain, dltest, data_processor = load_dataset(
    model_str=model_str,
    folder_path=data_path,
    train_batch_sizes=[1], test_batch_sizes=[1, 1],
    use_distributed=False,
    dataprocessor_dir='./')

model_inference = ModelInference(
    model=model,
    model_checkpoint_path=save_dir + 'best_model_snapshot_dict.pt',
    dataprocessor_path=f'./{model_str}/data_processor.pt',
    single_case_handling=single_case_handling)


l2loss = LpLoss(d=2, p=2, reductions='mean')

train_loss = l2loss
eval_losses={'l2': l2loss}

print(f"EPOCH: {model_inference.current_epoch}, LOSS: {model_inference.best_loss}")
model_inference.data_processor.training, model_inference.model.training

In [ ]:
from deepcardio.neuralop_core.utils import count_model_params
count_model_params(model)

In [ ]:
import json
from pathlib import Path
with open(Path(save_dir).joinpath('metrics_dict.json').as_posix(), 'r') as f:
    list_epoch_metrics = json.load(f)

epochs = []
training_losses = []
test_losses = []

for metrics_data in list_epoch_metrics:
    epochs.append(metrics_data['epoch'])
    training_losses.append(metrics_data['train_err'])
    test_losses.append(metrics_data['0_l2'])
    
plt.plot(epochs, training_losses, label="Training")
plt.plot(epochs, test_losses, label="Validation")
plt.yscale("log")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Training cases

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['case_id', 'loss'])
for i, sample in enumerate(dltrain[0]):
    model_inference.predict(sample)
    output = model_inference.output.to('cuda')
    train_loss = l2loss(output, **sample).item()
    torch.cuda.empty_cache()
    df.loc[i] = [int(sample['label'][0]), train_loss]
    print(f"Case ID: {int(sample['label'][0])}, Loss: {train_loss}")

df.to_csv(f'train_losses.csv', index='case_id')

plt.plot(df['loss'])
plt.show()

In [ ]:
print(torch.tensor(df['loss']).mean())
torch.topk(torch.tensor(df['loss']), k=20)

In [ ]:
sample = dltrain[0].dataset[df['loss'].idxmax()]
sample['label'], sample['n_pacings'], sample['y'].max(), l2loss(model_inference.predict(sample), **sample).item()
model_inference.write_xdmf()

# Val cases

In [ ]:
df = pd.DataFrame(columns=['case_id', 'loss'])
for i, sample in enumerate(dltest[0]):
    model_inference.predict(sample)
    output = model_inference.output.to('cuda')
    val_loss = l2loss(output, **sample).item()
    torch.cuda.empty_cache()
    df.loc[i] = [int(sample['label'][0]), val_loss]
    print(f"Case ID: {int(sample['label'][0])}, Loss: {val_loss}")
    
df.to_csv(f'val_losses.csv', index='case_id')

plt.plot(df['loss'])
plt.show()


In [ ]:
print(torch.tensor(df['loss']).mean())
torch.topk(torch.tensor(df['loss']), k=20)

# Test cases

In [ ]:
df = pd.DataFrame(columns=['case_id', 'max_tact', 'loss', 'noise1', 'noise5', 'noise10', 'noise20'])

for i, sample in enumerate(dltest[1]):
    for noise in [0., 0.01, 0.05, 0.1, 0.2]:
        sample['input_geom'] += torch.randn_like(sample['input_geom']) * (noise * sample['input_geom'].std())
        f0 = sample['a'][:, 2:5]
        sample['a'][:, 2:5] += torch.randn_like(f0) * (noise * f0.std())
        output = model_inference.predict(sample)
        test_loss = l2loss(output, **sample).item()
        torch.cuda.empty_cache()
        if noise == 0.:
            df.loc[
                i, ['case_id', 'max_tact', 'loss']
                ] = [
                    int(sample['label'][0]),
                    sample['y'].max().item(),
                    test_loss]
            print(f"Case ID: {int(sample['label'][0])}, Max TACT: {sample['y'].max().item()}, Loss: {test_loss}")
        elif noise == 0.01:
            df.loc[i, 'noise1'] = test_loss
        elif noise == 0.05:
            df.loc[i, 'noise5'] = test_loss
        elif noise == 0.1:
            df.loc[i, 'noise10'] = test_loss
        elif noise == 0.2:
            df.loc[i, 'noise20'] = test_loss
        torch.cuda.empty_cache()
df.to_csv(f'test_losses.csv', index='case_id')
plt.plot(df['loss'])
plt.show()

In [ ]:
print(torch.tensor(df['loss']).mean())
torch.topk(torch.tensor(df['loss']), k=20)

In [ ]:
sample = dltest[1].dataset[5371]
print(sample['label'], sample['n_pacings'], sample['y'].max(), l2loss(model_inference.predict(sample), **sample).item())
model_inference.write_xdmf()

In [ ]:
np.percentile(df['loss'], 99.5)

# Real-world LVs

In [ ]:
import glob
import os

cohort = 'YHC' # or 'HFC
folder_path = f'/mnt/research/compbiolab/Ehsan/DeepCardioSim/cardiac_models/electrophysio/data_realLV/{cohort}/'
YHC_losses = []
vtk_files = []
vtk_files.extend(glob.glob(os.path.join(folder_path, '*.vtk')))

df = pd.DataFrame(columns=['case_id', 'loss'])
for i, file in enumerate(vtk_files):
    model_inference.predict(file)
    output = model_inference.output.to('cuda')
    model_inference.case_ID = model_inference.case_ID[:-4]
    loss = l2loss(output, **model_inference.sample).item()
    torch.cuda.empty_cache()
    df.loc[i] = [int(model_inference.case_ID), loss]
df.to_csv(f'realLV_{cohort}_losses.csv', index='case_id')
plt.plot(df['loss'])
plt.show()

In [ ]:
print(torch.tensor(df['loss']).mean())
torch.topk(torch.tensor(df['loss']), k=20)

In [ ]:
model_inference.predict(f'/mnt/research/compbiolab/Ehsan/DeepCardioSim/cardiac_models/electrophysio/data_realLV/{cohort}/case19.vtk')
model_inference.case_ID = model_inference.case_ID[:-4]
model_inference.write_xdmf(
    inp_meshdir=f'/mnt/research/compbiolab/Ehsan/DeepCardioSim/cardiac_models/electrophysio/data_realLV/{cohort}/case19.vtk')

# RR cases

In [ ]:
from generate_interpolated_cases import refine_case_using_templates
df = pd.DataFrame(columns=['case_id', 'lossRR1', 'lossRR2'])

for i, sample in enumerate(dltest[1]):
    # if not np.isnan(df.loc[i]['lossRR2']):
    #     continue
    case_ID = sample['label'][0]
    npyfile = f'/mnt/research/compbiolab/Ehsan/DeepCardioSim/cardiac_models/electrophysio/data/npy/case{case_ID}_nplocs1.npy'
    if not Path(npyfile).exists():
        npyfile = f'/mnt/research/compbiolab/Ehsan/DeepCardioSim/cardiac_models/electrophysio/data/npy/case{case_ID}_nplocs2.npy'
        if not Path(npyfile).exists():
            raise ValueError('No default case-specific npy file found for setting up the mesh please pass the input mesh directory and try again.')
    npydata = np.load(npyfile)
    points_coarse = npydata[:, :3]

    coarse_pd = {
        'ploc_bool': npydata[:, 3:4],
        'D_iso': npydata[:, 4:5],
        'ef': npydata[:, 5:8],
        'activation_time': npydata[:, 8:9]
    }

    mesh_RR1 = refine_case_using_templates(points_coarse, coarse_pd, level='RR1')
    mesh_RR1.save(f'./case{case_ID}_RR1.vtk')
    model_inference.predict(inp = f'./case{case_ID}_RR1.vtk')
    Path(f'./case{case_ID}_RR1.vtk').unlink()
    output = model_inference.output.to('cuda')
    model_inference.case_ID = model_inference.case_ID[:-8]
    model_inference.sample['label'] = model_inference.case_ID
    loss_RR1 = l2loss(output, **model_inference.sample).item()
    torch.cuda.empty_cache()

    mesh_RR2 = refine_case_using_templates(points_coarse, coarse_pd, level='RR2')
    mesh_RR2.save(f'./case{case_ID}_RR2.vtk')
    with torch.no_grad(), torch.autocast(device_type='cuda'):
        model_inference.predict(inp = f'./case{case_ID}_RR2.vtk')
    
    output = model_inference.output.to('cuda')
    model_inference.case_ID = model_inference.case_ID[:-8]
    model_inference.sample['label'] = model_inference.case_ID
    loss_RR2 = l2loss(output, **model_inference.sample).item()
    torch.cuda.empty_cache()
    Path(f'./case{case_ID}_RR2.vtk').unlink()

    df.loc[i] = [
        int(model_inference.sample['label']),
        loss_RR1,
        loss_RR2]
    print(f'{i}:, {loss_RR1}, {loss_RR2}')

    if (i+1) % 100 == 0:
        df.to_csv(f'refined_losses.csv', index='case_id')
    
df.to_csv(f'refined_losses.csv', index='case_id')

# CRT workflow

In [ ]:
# Inverse problem
import meshio
sample = dltest[1].dataset[2185] # case 48245
sample = sample.to(model_inference.device)
geometry = meshio.read('/mnt/home/naghavis/Documents/Research/DeepCardioSim/deepcardio/LVmean/LV_mean.vtk')
npy_data = np.load('/mnt/research/compbiolab/Ehsan/DeepCardioSim/cardiac_models/electrophysio/data/npy/case48245_nplocs1.npy')
geometry.points = npy_data[:, :3]
pmtr_coord = np.concatenate((
        geometry.point_data['x_l'],
        geometry.point_data['x_c'],
        geometry.point_data['x_t']), axis=1)

measured_act = torch.tensor(
    np.array(meshio.read('sample_crt.vtk').point_data['t_act'], dtype=np.float64),
    dtype=sample['y'].dtype,
    device=sample['y'].device)

min_act_loc = np.argmin(measured_act.cpu().numpy())

x0 = [0.5, pmtr_coord[min_act_loc, 0], pmtr_coord[min_act_loc, 1]]

def optimization_function_wrapper(x):
    """
    This wraps the function for the inverse modeling
    optimization problem.

    Input:
    - x: a numpy array represents [D_iso, x_l, x_c]

    Output:
    - loss: The loss of the optimization problem, returned as a
    scalar numpy value
    """
    ploc_pmtr = [x[1], x[2], 1]

    min_loc = np.argmin(np.linalg.norm(pmtr_coord - np.array(ploc_pmtr), axis=1))
    ploc_xyz = np.array([geometry.points[min_loc]])


    output = model_inference.predict(sample, Diso=x[0], plocs=ploc_xyz)

    loss = l2loss(output, sample['y'])
    return loss.item()

from scipy.optimize import differential_evolution

bounds = [(0.1, 2.), (0.1, .9), (-3., 3.)]

result = differential_evolution(optimization_function_wrapper,
                                        bounds,
                                        seed=45,
                                        popsize=45,
                                        strategy='best1bin',
                                        x0=x0,
                                        maxiter=100,
                                        disp=True)

print(result.x)

print((model_inference.output - sample['y']).abs().max(), (model_inference.output - sample['y']).abs().mean())

In [ ]:
# Optimization problem
def optimization_function_wrapper(x):
    """
    This wraps the function for the inverse modeling
    optimization problem.

    Input:
    - x: a numpy array represents [x_l2, x_c2]

    Output:
    - loss: The loss of the optimization problem, returned as a
    scalar numpy value
    """
    plocs_pmtr = [[result.x[1], result.x[2], 1], [x[0], x[1], 1]]
    plocs_xyz = []
    for ploc_ in plocs_pmtr:
        min_loc = np.argmin(np.linalg.norm(pmtr_coord - np.array(ploc_), axis=1))
        ploc_xyz = np.array([geometry.points[min_loc]])
        plocs_xyz.append(ploc_xyz)

    plocs_xyz = np.array(plocs_xyz)


    output = model_inference.predict(sample, Diso=result.x[0], plocs=plocs_xyz)

    return output.max().item()

bounds = [(0.1, .9), (-3., 3.)]

result_CRT = differential_evolution(
    optimization_function_wrapper,bounds,
    seed=45,
    popsize=30,
    strategy='best1bin',
    maxiter=100,
    disp=True)

result_CRT.x, result_CRT.fun

In [ ]:
plocs_xyz = []

plocs_pmtr = [[result.x[1], result.x[2], 1], [result_CRT.x[0], result_CRT.x[1], 1]]

for ploc_ in plocs_pmtr:
    min_loc = np.argmin(np.linalg.norm(pmtr_coord - np.array(ploc_), axis=1))
    ploc_xyz = np.array([geometry.points[min_loc]])
    plocs_xyz.append(ploc_xyz)

plocs_xyz = np.array(plocs_xyz)

plocs_xyz

In [ ]:
output = model_inference.predict(sample, Diso = result.x[0], plocs=plocs_xyz)
model_inference.write_xdmf()
output.max()